# South Sudan Data Layers

This notebook is used to prepare the data layers for South Sudan. The data layers will be processed and upload to Mapbox.

## Data Hierarchy

The data is organized in the following hierarchy:

- [This](https://docs.google.com/spreadsheets/d/1RdJCjygAiWu2zBMGRF0ayigzrA2WhaObWMNdlkllgSQ/edit?usp=sharing) is the link to the data hierarchy spreadsheet.

## Data Access
The data is stored in the following Google Cloud Storage bucket:
- https://console.cloud.google.com/storage/browser/wbhydross_deliverables


## Setup

### Library import


In [1]:
# imports
import json
import sys
from pathlib import Path
from pprint import pprint

# Include local library paths if you have ../src/utils.py
sys.path.append("../src/")
sys.path.append("../src/datasets")
sys.path.append("../src/datasets/factory")

from datasets.datasets import dataset_database
from helpers.mapbox_uploader import upload_to_mapbox
from helpers.mbtiles_converter import MBTilesConverter
from helpers.raster_tiles import RasterTiles
from helpers.settings import get_settings
from helpers.tippecanoe import mbtile_generation
from raster_processor import QgsStyledRasterProcessor
from tqdm import tqdm

In [2]:
# Load settings with environment variables
settings = get_settings()

# Data Acquisition

## Dataset information

In [3]:
datasets = dataset_database.datasets()
pprint(datasets)

{'Agricultural drought exposure': <datasets.datasets.Dataset object at 0x7f9629a57c80>,
 'Agricultural drought hazard': <datasets.datasets.Dataset object at 0x7f9629a57c50>,
 'Boundaries': <datasets.datasets.Dataset object at 0x7f9629a57c20>,
 'Contextual layers': <datasets.datasets.Dataset object at 0x7f9629a57bf0>,
 'EO-based flood exposure': <datasets.datasets.Dataset object at 0x7f9629a57b30>,
 'EO-based flood hazard': <datasets.datasets.Dataset object at 0x7f9629a57bc0>,
 'Hydrographic data': <datasets.datasets.Dataset object at 0x7f9629a57b60>,
 'Hydrometeorological Data': <datasets.datasets.Dataset object at 0x7f9629a57b90>,
 'Meteorological drought exposure': <datasets.datasets.Dataset object at 0x7f9629a57ad0>,
 'Meteorological drought hazard': <datasets.datasets.Dataset object at 0x7f9629a57b00>,
 'Model-based flood exposure': <datasets.datasets.Dataset object at 0x7f9629a57a70>,
 'Model-based flood hazard': <datasets.datasets.Dataset object at 0x7f9629a57aa0>,
 'Populated in

## Static datasets
### Create layers

In [4]:
datasets_list = [
    "Model-based flood hazard",
    "EO-based flood hazard",
    "Meteorological drought hazard",
    "Agricultural drought hazard",
    "Model-based flood exposure",
    "EO-based flood exposure",
    "Meteorological drought exposure",
    "Agricultural drought exposure",
]

# Load datasets_dict
with open("../data/processed/datasets_dict.json", "r") as file:
    datasets_dict = json.load(file)

for dataset_name in tqdm(datasets_list):
    print(dataset_name)
    shortened_dataset_name = "".join(word[0] for word in dataset_name.split()).upper()

    dataset = datasets.get(dataset_name)
    layers = dataset.layers()
    dataset_layers = datasets_dict.get(dataset_name, {})
    for layer_name, layer in tqdm(layers.items()):
        if layer_name not in dataset_layers:
            layer_name_lower = layer_name.lower().replace(" - ", " ").replace(" ", "_")
            file_name = f"{shortened_dataset_name}_{layer_name_lower}"
            print(f"{shortened_dataset_name}_{layer_name_lower}")
            dataset_layers[layer_name] = file_name
            datasets_dict[dataset_name] = dataset_layers
            if layer.type == "raster" and layer.format == "GeoTIFF":
                # Define the output path
                base_path = Path("../data/processed/RasterLayers")
                output_path = base_path / Path(file_name).with_suffix(".tif")
                if layer.styles:
                    # QML file
                    qml_file = layer.styles
                    # Style raster and save it as Cloud Optimized GeoTIFF
                    QgsStyledRasterProcessor(layer.url, qml_file, output_path).process()

            elif layer.type == "vector":
                # Define the output path
                base_path = Path("../data/processed/VectorLayers")
                output_path = base_path / Path(file_name).with_suffix(".mbtiles")
                # Generate MBTile
                df = layer.get_data()
                mbtile_generation(df, output_path)

            # Save datasets_dict to a file
            with open("../data/processed/datasets_dict.json", "w") as f:
                json.dump(datasets_dict, f)

  0%|          | 0/8 [00:00<?, ?it/s]

Model-based flood hazard


100%|██████████| 10/10 [00:00<00:00, 179243.76it/s]


EO-based flood hazard


100%|██████████| 2/2 [00:00<00:00, 43690.67it/s]


...

100%|██████████| 10/10 [00:00<00:00, 186413.51it/s]


Meteorological drought exposure


100%|██████████| 8/8 [00:00<00:00, 184365.01it/s]


Agricultural drought exposure


100%|██████████| 8/8 [00:00<00:00, 503.84it/s]


### Raster layers

#### Convert GeoTIFFs to Tiles

In [4]:
datasets_list = [
    "Model-based flood hazard",
    "EO-based flood hazard",
    "Meteorological drought hazard",
    "Agricultural drought hazard",
    "Model-based flood exposure",
    "EO-based flood exposure",
    "Meteorological drought exposure",
    "Agricultural drought exposure",
]

# Load datasets_dict
with open("../data/processed/datasets_dict_raster_tiles.json", "r") as file:
    datasets_dict = json.load(file)

for dataset_name in tqdm(datasets_list):
    print(dataset_name)
    shortened_dataset_name = "".join(word[0] for word in dataset_name.split()).upper()

    dataset = datasets.get(dataset_name)
    layers = dataset.layers()
    dataset_layers = datasets_dict.get(dataset_name, {})
    for layer_name, layer in tqdm(layers.items()):
        if layer_name not in dataset_layers:
            layer_name_lower = layer_name.lower().replace(" - ", " ").replace(" ", "_")
            file_name = f"{shortened_dataset_name}_{layer_name_lower}"
            print(f"{shortened_dataset_name}_{layer_name_lower}")
            dataset_layers[layer_name] = file_name
            datasets_dict[dataset_name] = dataset_layers
            if layer.type == "raster" and layer.format == "GeoTIFF":
                # Define the output path
                base_path = Path("../data/processed/RasterLayers")
                input_path = base_path / Path(file_name).with_suffix(".tif")
                output_folder = Path("../data/processed/RasterTiles") / Path(file_name)
                output_folder.mkdir(parents=True, exist_ok=True)

                # Convert GeoTIFF to Tiles
                raster_tiles = RasterTiles(
                    input_path, output_folder, min_z=4, max_z=12, engine="rasterio"
                )
                raster_tiles.create()

            # Save datasets_dict to a file
            with open("../data/processed/datasets_dict_raster_tiles.json", "w") as f:
                json.dump(datasets_dict, f)

  0%|          | 0/8 [00:00<?, ?it/s]

Model-based flood hazard


MFH_fluvial_flood_depth_10-yr_rp
Creating tiles ...


MFH_fluvial_flood_depth_100-yr_rp
Creating tiles ...


MFH_fluvial_flood_depth_20-yr_rp
Creating tiles ...


...

MFH_pluvial_flood_depth_50-yr_rp
Creating tiles ...


 12%|█▎        | 1/8 [36:50<4:17:50, 2210.03s/it]

EO-based flood hazard


EFH_flood_extent_(2017-2022)
Creating tiles ...


EFH_flood_max_frequency_(2017-2022)
Creating tiles ...


#### Convert GeoTIFFs to MBTiles

In [4]:
datasets_list = [
    "Model-based flood hazard",
    "EO-based flood hazard",
    "Meteorological drought hazard",
    "Agricultural drought hazard",
    "Model-based flood exposure",
    "EO-based flood exposure",
    "Meteorological drought exposure",
    "Agricultural drought exposure",
]

# Load datasets_dict
with open("../data/processed/datasets_dict_mbtiles.json", "r") as file:
    datasets_dict = json.load(file)

for dataset_name in tqdm(datasets_list):
    print(dataset_name)
    shortened_dataset_name = "".join(word[0] for word in dataset_name.split()).upper()

    dataset = datasets.get(dataset_name)
    layers = dataset.layers()
    dataset_layers = datasets_dict.get(dataset_name, {})
    for layer_name, layer in tqdm(layers.items()):
        if layer_name not in dataset_layers:
            layer_name_lower = layer_name.lower().replace(" - ", " ").replace(" ", "_")
            file_name = f"{shortened_dataset_name}_{layer_name_lower}"
            print(f"{shortened_dataset_name}_{layer_name_lower}")
            dataset_layers[layer_name] = file_name
            datasets_dict[dataset_name] = dataset_layers
            if layer.type == "raster" and layer.format == "GeoTIFF":
                # Define the output path
                base_path = Path("../data/processed/RasterLayers")
                input_path = base_path / Path(file_name).with_suffix(".tif")
                output_path = base_path / Path(file_name).with_suffix(".mbtiles")

                # Convert GeoTIFF to MBTiles
                MBTilesConverter.convert(input_path, output_path)

            # Save datasets_dict to a file
            with open("../data/processed/datasets_dict_mbtiles.json", "w") as f:
                json.dump(datasets_dict, f)

  0%|          | 0/8 [00:00<?, ?it/s]

Model-based flood hazard


MFH_fluvial_flood_depth_10-yr_rp


In [ ]:
base_path = Path("../data/processed/RasterLayers")
file_name = "MFH_fluvial_flood_depth_10-yr_rp"
input_path = base_path / Path(file_name).with_suffix(".tif")
output_path = base_path / Path(file_name).with_suffix(".mbtiles")
MBTilesConverter.convert(input_path, output_path)

### Upload layers to Mapbox

In [4]:
datasets_list = [
    "Model-based flood hazard",
    "EO-based flood hazard",
    "Meteorological drought hazard",
    "Agricultural drought hazard",
    "Model-based flood exposure",
    "EO-based flood exposure",
    "Meteorological drought exposure",
    "Agricultural drought exposure",
]

# Load datasets_dict
with open("../data/processed/datasets_dict_mapbox.json", "r") as file:
    datasets_dict = json.load(file)

for dataset_name in tqdm(datasets_list):
    print(dataset_name)
    shortened_dataset_name = "".join(word[0] for word in dataset_name.split()).upper()

    dataset = datasets.get(dataset_name)
    layers = dataset.layers()
    dataset_layers = datasets_dict.get(dataset_name, {})
    for layer_name, layer in tqdm(layers.items()):
        if layer_name not in dataset_layers:
            layer_name_lower = layer_name.lower().replace(" - ", " ").replace(" ", "_")
            file_name = f"{shortened_dataset_name}_{layer_name_lower}"
            print(f"{shortened_dataset_name}_{layer_name_lower}")
            dataset_layers[layer_name] = file_name
            datasets_dict[dataset_name] = dataset_layers
            if layer.type == "raster" and layer.format == "GeoTIFF":
                # Define the output path
                base_path = Path("../data/processed/RasterLayers")
                output_path = base_path / Path(file_name).with_suffix(".tif")

            elif layer.type == "vector":
                # Define the output path
                base_path = Path("../data/processed/VectorLayers")
                output_path = base_path / Path(file_name).with_suffix(".mbtiles")

            # Upload to Mapbox
            upload_name = upload_to_mapbox(
                output_path,
                file_name,
                settings.MAPBOX_USER,
                settings.MAPBOX_TOKEN,
            )

            # Save datasets_dict to a file
            with open("../data/processed/datasets_dict_mapbox.json", "w") as f:
                json.dump(datasets_dict, f)

  0%|          | 0/8 [00:00<?, ?it/s]

Model-based flood hazard


100%|██████████| 10/10 [00:00<00:00, 229196.94it/s]


EO-based flood hazard


100%|██████████| 2/2 [00:00<00:00, 38304.15it/s]


...

INFO:helpers.mapbox_uploader:Uploading to Mapbox...


MDH_combined_spi_and_spei_indices


INFO:helpers.mapbox_uploader:Uploading to S3...
 25%|██▌       | 2/8 [00:01<00:04,  1.22it/s]


upload: ../data/processed/RasterLayers/MDH_combined_spi_and_spei_indices.tif to s3://tilestream-tilesets-production/46/_pending/vk61klateom19wy2ny0vo12mc/wims


KeyboardInterrupt: 

## Animated raster data

In [19]:
raster_layers = {}
for dataset_name, dataset in datasets.items():
    layers = dataset.layers()
    dataset_layers = {}
    for layer_name, layer in layers.items():
        if layer.type == "raster" and layer.format == "Zarr":
            dataset_layers[layer_name] = layer
    if dataset_layers:
        raster_layers[dataset_name] = dataset_layers

raster_layers

{'Hydrometeorological Data': {'Evapotranspiration': <datasets.datasets.Layer at 0x7f99970f5eb0>,
  'Precipitation': <datasets.datasets.Layer at 0x7f99970f5fa0>,
  'Soil moisture': <datasets.datasets.Layer at 0x7f99970f4b60>,
  'Temperature': <datasets.datasets.Layer at 0x7f99970f6c60>}}

In [30]:
layer = raster_layers["Hydrometeorological Data"]["Evapotranspiration"]
ds = layer.get_data()

Loading Zarr data from gs://wbhydross_deliverables/D3-Database/02- Meteorological datasets/Evapotranspiration-WaPOR/WBHYDROSSD_WaPOR_Evapotranspiration_100m_SouthSudan_2023_20240220.zarr...


In [32]:
ds

<xarray.Dataset> Size: 12GB
Dimensions:      (months: 12, y: 9728, x: 12800)
Coordinates:
  * months       (months) int32 48B 1 2 3 4 5 6 7 8 9 10 11 12
  * x            (x) float64 102kB 23.83 23.83 23.83 23.83 ... 36.33 36.33 36.33
  * y            (y) float64 78kB 12.7 12.7 12.7 12.7 ... 3.203 3.202 3.201 3.2
Data variables:
    AETI         (months, y, x) float64 12GB dask.array<chunksize=(1, 9728, 12800), meta=np.ndarray>
    spatial_ref  int32 4B ...

In [33]:
data = ds.isel(months=0)
data = data.rio.write_crs("EPSG:4326")
# Save as COG
output_path = "../data/processed/evapotranspiration.tif"
data.rio.to_raster(output_path, driver="COG")